In [1]:
import pandas as pd
import numpy as np

In [2]:
import pickle

In [3]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [4]:
from sklearn.metrics import auc, roc_auc_score, roc_curve, accuracy_score,  recall_score, precision_score
from regression_tools.dftransformers import (
    ColumnSelector, Identity, FeatureUnion, MapFeature, Intercept)

In [5]:
from my_metr import plot_roc_curve

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
from pymongo import MongoClient
import time
from bs4 import BeautifulSoup

mc = MongoClient()
db  = mc['raw_restaurants']
rv_s = db['reviews_scrap']

In [7]:
colClust_n = ['cl_t0', 'cl_t1', 'cl_t2', 'cl_t3', 'cl_t4', 'cl_t5',  'cl_t7',
       'cl_t8', 'cl_t9', 'cl_t10', 'cl_t11', 'cl_t12', 'cl_t13', 'cl_t14',
        'cl_t16', 'cl_t17']

In [ ]:
#numbers_list = pickle.load(list_unpickle)

In [8]:
clust_model_rf ={}
clust_model_gr = {}

In [10]:
for col in colClust_n:
    filename_gr = 'grboost/mgr_'+col+'.pkl'
    with open(filename_gr,'rb') as f:
        clust_model_gr[col]= pickle.load( f)
    filename_rf = 'rforest/mrf_'+col+'.pkl'
    with open(filename_rf,'rb') as f:
        clust_model_rf[col] =pickle.load( f)

In [11]:
clust_model_gr['cl_t3']

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=3,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=300,
              presort='auto', random_state=179, subsample=0.8, verbose=0,
              warm_start=False)

In [13]:
user_cl = pd.read_csv('user_from_scr_clidf18.csv')

In [14]:
rest = db['restaurants']

rest_df = pd.DataFrame(list(rest.find({})))

### надо почистить дату

In [16]:
clust_model_gr['cl_t3'].predict_proba(rest_df)

ValueError: could not convert string to float: 'https://www.yelp.com/biz/mcdonalds-sedro-woolley-2?adjust_creative=KIqjD5y9rnzq9ev6ruaTHQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=KIqjD5y9rnzq9ev6ruaTHQ'